In [10]:
import sys
import pandas as pd
import numpy as np
sys.path.append('../src')
from spcqe.functions import fit_quantiles

In [11]:
def process_excel_file(file_name, year):
    # Check if the year is a leap year
    is_leap_year = (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0)
    
    # Define the start and end of your time range
    start = pd.Timestamp(f'{year}-01-01 00:00:00')
    end = pd.Timestamp(f'{year}-12-31 23:00:00')
    
    # Create an hourly time index between start and end
    time_index = pd.date_range(start, end, freq='H')
    
    # Read the Excel file and select columns
    df = pd.read_excel(file_name, sheet_name='RI', usecols=['RT_Demand', 'Dry_Bulb', 'RT_LMP'])
    
    # Assign your new time index to your dataframe
    df.index = time_index
    
    # If it's a leap year, remove the row corresponding to February 29
    if is_leap_year:
        feb_29 = pd.Timestamp(f'{year}-02-29 00:00:00')
        df = df[(df.index < feb_29) | (df.index >= feb_29 + pd.Timedelta(days=1))]
    
    # Convert columns to numpy arrays
    y1 = df['RT_Demand'].to_numpy()
    y2 = df['Dry_Bulb'].to_numpy()
    y3 = df['RT_LMP'].to_numpy()
    
    return y1, y2, y3

#years = [2019, 2020, 2021, 2022]
years = [2021, 2022]
y1_list, y2_list, y3_list = [], [], []

for year in years:
    file_name = f'./fixtures/{year}_smd_hourly.xlsx'  # Assuming your file name format
    y1, y2, y3 = process_excel_file(file_name, year)
    y1_list.append(y1)
    y2_list.append(y2)
    y3_list.append(y3)

# Stack the arrays in each list column-wise
y1 = np.column_stack(y1_list)
y2 = np.column_stack(y2_list)
y3 = np.column_stack(y3_list)

In [12]:
percentiles = [2,10,20,30,40,50,60,70,80,90,98]
P1 = int(365*24)
P2 = int(7*24)
P3 = int(24)
K = 3
l = 0.1
y = y1.flatten('F')
Q1,Q_extended1,GQ_extended1 = fit_quantiles(y.reshape(-1,1), K, P1, P2, P3, l, percentiles)

                                     CVXPY                                     
                                     v1.3.1                                    
(CVXPY) Jul 19 01:36:58 PM: Your problem has 1397 variables, 1 constraints, and 0 parameters.
(CVXPY) Jul 19 01:36:58 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 19 01:36:58 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 19 01:36:58 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 19 01:36:58 PM: Compiling problem (target solver=MOSEK).
(CVXPY) Jul 19 01:36:58 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing

In [13]:
Q1.shape

(17520, 11)